In [ ]:
# !pip install mecab-python3
#These wheels include a copy of the MeCab library, but not a dictionary. 
#In order to use MeCab you'll need to install a dictionary. unidic-lite is a good one to start with:
# !pip install unidic-lite

In [ ]:
# normalization tool
# !pip install neologdn

In [ ]:
# !pip install openpyxl

In [ ]:
# # To be able to see Japanese!
# !pip install japanize_matplotlib

everything was deleted what a christmas miracle....
lets start again ...


## Librarys

In [ ]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

import os
import pandas as pd

# Preprocessing
import MeCab
import neologdn
import collections
from nltk import FreqDist
from nltk.corpus import stopwords

# Visualization
import matplotlib.pyplot as plt
import japanize_matplotlib
#import seaborn as sns # REMINDER: make sure to remove if not using!

In [ ]:
file_name = 'Combined_85K_150.xlsx'
df = pd.read_excel(r'/home/samuelhenderson/code/simply-japanese/data/2_RawData/' + file_name)
df.drop(columns=['#英語(原文)'], inplace=True)
df.rename(columns={"#日本語(原文)": "original", "#やさしい日本語": "simplified"}, inplace=True)
df.head()

In [ ]:
# cosine similarity 
# X= df['original']
# y= df['simplified']

# cosine_similarity(X, y)

In [ ]:
#testing out stuff from github repository : 
# https://github.com/ZHAOTING/Japanese-sentence-similarity-scorer

In [ ]:
# !python -m unidic download

In [ ]:
# #fast text
# import fasttext
# import fasttext.util

In [ ]:
# fasttext.util.download_model('jp', if_exists='ignore')  # English
# ft = fasttext.load_model('cc.en.300.bin')

## SumEval & Bleu caluculator (Sacrebleu)
-------------------------- can come back to this but WER sounds more promising... --------------------------

https://github.com/chakki-works/sumeval#welcome-contribution-tada

In [ ]:
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download en_core_web_sm

Trying to get the right dependency for the sumeval & Bleu caluculator (Sacrebleu)

In [ ]:
 # pip install sumeval==0.1.5

In [ ]:
# pip uninstall sumeval

In [ ]:
# pip install janome

In [ ]:
import sacrebleu

In [ ]:
from sacrebleu import corpus_bleu, DEFAULT_TOKENIZER

In [ ]:
type(df.iloc[0][0])

In [ ]:
# from sumeval.metrics.bleu import BLEUCalculator
# # https://github.com/chakki-works/sumeval
# from janome.tokenizer import Tokenizer

# # bleu calculator for english
# # bleu = BLEUCalculator()
# # score = bleu.bleu("I am waiting on the beach",
# #                   "He is walking on the beach")

# # bleu calculator for Japanese
# bleu_ja = BLEUCalculator(lang="ja")
# score_ja = bleu_ja.bleu(df.iloc[0][0], df.iloc[0][1])
# tokenizer_ja()

In [ ]:
# from sumeval.metrics.bleu import BLEUCalculator

# bleu = BLEUCalculator()
# score = bleu.bleu("I am waiting on the beach",
#                   "He is walking on the beach")

# # bleu_ja = BLEUCalculator(lang="ja")
# # score_ja = bleu_ja.bleu("私はビーチで待ってる", "彼がベンチで待ってる")

In [ ]:
# from sacrebleu import ja_mecab

In [ ]:
# bleu = BLEUCalculator()

In [ ]:
# df.iloc[0][0]

In [ ]:
# from sacrebleu.metrics import BLEU, CHRF, TER

# bleu_ja = BLEUCalculator(lang="ja")
# score_ja = bleu_ja.bleu(df.iloc[0][0], df.iloc[0][1])


In [ ]:
# import sacrebleu

In [ ]:
sacrebleu.info()
# https://github.com/chakki-works/sumeval#welcome-contribution-tada

In [ ]:
# from sacrebleu.metrics import BLEU, CHRF, TER

# refs = [ # First set of references
#           ['The dog bit the man.', 'It was not unexpected.', 'The man bit him first.'],
#           # Second set of references
#           ['The dog had bit the man.', 'No one was surprised.', 'The man had bitten the dog.'],
#         ]
# sys = ['The dog bit the man.', "It wasn't surprising.", 'The man had just bitten him.']

# bleu = BLEU()

# bleu.corpus_score(sys, refs)
# # BLEU = 48.53 82.4/50.0/45.5/37.5 (BP = 0.943 ratio = 0.944 hyp_len = 17 ref_len = 18)

# bleu.get_signature()
# # nrefs:2|case:mixed|eff:no|tok:13a|smooth:exp|version:2.0.0

# chrf = CHRF()

# chrf.corpus_score(sys, refs)
# # chrF2 = 59.73

In [ ]:
# !pip list

## WER (Word Error Rate) and Edit Distance
WER - https://en.m.wikipedia.org/wiki/Word_error_rate 
WER= {S+D+I}/{N}={S+D+I}/{S+D+C}
where:

**S** is the number of substitutions,

**D** is the number of deletions,

**I** is the number of insertions,

**C** is the number of correct words,

**N** is the number of words in the reference (N=S+D+C)

Edit Distance (Levenshtein distance) https://en.m.wikipedia.org/wiki/Edit_distance
Takes two either been an insert/remove/replace of characters.

i.e. 
Input:   str1 = “geek”, str2 = “gesek”

Output:  1

Explanation: We can convert str1 into str2 by inserting a ‘s’.

In [ ]:
df

In [ ]:
# A WER function I found online.
# Seems be going off legth of characters in simplification process,
# But in a fair few of the conversions the phrases get longer.

def wer(ref, hyp ,debug=True):
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
 
    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3
    DEL_PENALTY = 1
    INS_PENALTY = 1
    SUB_PENALTY = 1
    
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL
    
    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS
    
    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1
                 
                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL
                 
    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        # print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("OK\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    # if debug:
    #     lines = reversed(lines)
    #     for line in lines:
    #         print(line)
            
        # Removing all the extra bits we dont need at this second

        # print("#correct words " + str(numCor))
        # print("#substitutions " + str(numSub))
        # print("#deleltions " + str(numDel))
        # print("#insertions " + str(numIns))
    return (numSub + numDel + numIns) / (float) (len(r))
    
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
    print({'WER':wer_result, 'numCorrect':numCor, 'numSubstitutions':numSub, 'numInserts':numIns, 'numDelete':numDel, "numCount": len(r)})

In [ ]:
# Testing first sample
wer(df.iloc[0][0], df.iloc[0][1])

In [ ]:
df.iloc[0][0]

In [ ]:
# Testing Sample 2
wer(df.iloc[2][0], df.iloc[2][1])

In [ ]:
# Checking if the length of the characters are of different lengths for the 2nd Sample.
len(df.iloc[2][0]), len(df.iloc[2][1])

In [ ]:
print("Finding the samples with differences in characters...")
for i in range(150):
    if len(df.iloc[i][0])> len(df.iloc[i][1]):
        print(i, len(df.iloc[i][0]), len(df.iloc[i][1]))

In [ ]:
# Testing on the original to simplified sentences with the WER metric
# returing sentence as whole string so not able to give us much insight
# Only returning if it is a WER of 0 if it is the same,
# Or returning a WER of 1.0 if it is differnt

# 
print('Testing WER on simplified sentences with smaller outputs than input...')
print('')
for i in range(150):    
    if len(df.iloc[i][0])> len(df.iloc[i][1]):
        print('Sample', i+1)
        print('Difference in Characters = ', len(df.iloc[i][0]) - len(df.iloc[i][1]))
        print(len(df.iloc[i][0]) - len(df.iloc[i][1]))
        wer(df.iloc[i][0], df.iloc[i][1])
        print('*******************************************************************************')

In [ ]:
# Just checking on a string in Enlgish how it works...
# i = 10
wer('I love to dance and sing', 'I love singing and i love to do dancing')

In [ ]:
df.iloc[84]

In [ ]:
#  Making a function to iterate over the whole Data set to to see what it spits out

#Creating a Function to split all characters to determine how many have been changed.
def wer_jp(original, simplified):       
    ori = ''
    simpi = ''
    wer_score = []
    for i in original:
        ori += i + ' '
    for i in simplified:
        simpi += i + ' '
    # if round(wer(ori, simpi),3):
    print('WER Score ', round(wer(ori, simpi),3))
        # df.insert(2, "WER Score", round(wer(ori, simpi), True))
    return round(wer(ori, simpi),3)
    return wer_score   

# for i in df.index:
#     original_text = df.iloc[i][0] 
#     simplified_text = df.iloc[i][1]
#     print('******************')
#     print('Testing sample ', i+1)
#     # print('******************')
#     wer_jp(original_text, simplified_text)
    

In [ ]:
df['WER_score'] = wer_score

In [ ]:
df

In [ ]:
import seaborn
seaborn.histplot(df['WER_score'], bins=150)

In [ ]:
def wer_score(predicted, simplified, debug=True):
    '''
    Compares the simplified ML prediction of a given text to the pre-existing simplified
    text given with the dataframe.
    Using the WER (word error rate) algorithm. 
    Adds the WER score as a new column to the Dataframe
    '''
    r = predicted.split()
    h = simplified.split()
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
 
    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3
    DEL_PENALTY = 1
    INS_PENALTY = 1
    SUB_PENALTY = 1
    
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS
    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1
                 
                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL
                 
    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    
    if debug:
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("OK\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    
    return (numSub + numDel + numIns) / (float) (len(r))
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)

def wer_jp(original, simplified):       
    ori = ''
    simpi = ''
    wer_score = []
    for i in original:
        ori += i + ' '
    for i in simplified:
        simpi += i + ' '
    print('WER Score ', round(wer(ori, simpi),3))
    return round(wer(ori, simpi),3)
    return wer_score
    
def evaluate_wer_score(df):
    wer_list = []
    for i in df.index:
        original_text = df.iloc[i][0] 
        simplified_text = df.iloc[i][1]
        wer_list.append(wer_jp(original_text, simplified_text))
    df['WER_score'] = wer_list

In [ ]:
# evaluating(df)

# Tensorflow Seq2Seq
Having a play

In [ ]:
# !pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import unicodedata

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from nltk.translate.bleu_score import sentence_bleu

In [ ]:
def preprocess(text):
    # remove diacritics (accents)
    text = ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    # remove punctuation and digits
    text = re.sub('[%s]' % re.escape(string.punctuation + string.digits), '', text)
    # remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()
    return text

def prepare_text_dataset(texts, max_vocab_size=10000, max_seq_len=100):
    vectorizer = TextVectorization(max_tokens=max_vocab_size, output_mode='int', output_sequence_length=max_seq_len)
    vectorizer.adapt(texts)
    return vectorizer

def prepare_seq2seq_dataset(texts, vectorizer):
    input_texts = ['<start> ' + text for text in texts]
    target_texts = [text + ' <end>' for text in texts]
    input_seqs = vectorizer(np.array(input_texts))[:, :-1]
    target_seqs = vectorizer(np.array(target_texts))[:, 1:]
    return (input_seqs, target_seqs)


#Preprocess text and prepare the datasetfor seq2seq transformer
df['text'] = df['text'].apply(preprocess)
max_vocab_size = 10000
max_seq_len = 100
text_vectorizer = prepare_text_dataset(df['text'], max_vocab_size, max_seq_len)
input_seqs, target_seqs = prepare_seq2seq_dataset(df['text'], text_vectorizer)


# Let's create the seq2seq transformer model:
embedding_dim = 256
num_heads = 8
ff_dim = 512
dropout_rate = 0.1
input_vocab_size = max_vocab_size + 2
target_vocab_size = max_vocab_size + 2

inputs = keras.Input(shape=(None,), dtype="int64")
targets = keras.Input(shape=(None,), dtype="int64")

encoder = keras.layers.Encoder(input_vocab_size, embedding_dim, num_heads, ff_dim, dropout_rate)
decoder = keras.layers.Decoder(target_vocab_size, embedding_dim, num_heads, ff_dim, dropout_rate)
outputs = decoder(encoder(inputs), targets)

model = keras.Model([inputs, targets], outputs)


# Let's compile the model and train it on the dataset:
batch_size = 64
epochs = 10

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit([input_seqs, target_seqs], target_seqs, batch_size=batch_size, epochs=epochs, validation_split=0.2)

def predict_simplified_text(model, text, vectorizer, max_len=100):
    text = preprocess(text)

In [ ]:
#  BLEU Score
from nltk.translate.bleu_score import sentence_bleu

# Example reference and candidate sentences
reference = ['私は犬が好きです', 'あなたは猫が好きですか']
candidate = '私は猫が好きです'

# Compute the BLEU score for the candidate sentence
# score = sentence_bleu(reference, candidate)
# print('BLEU score:', score)

score = sentence_bleu(df.iloc[0][0], df.iloc[0][1])
print('BLEU score:', score)

In [ ]:
# Preprossesssing
import MeCab
import sudachipy
from pyknp import Juman
import re
import fasttext

# Set up MeCab tokenizer
mecab = MeCab.Tagger('-Owakati')

# Set up SudachiPy tokenizer
sudachi = sudachipy.Dictionary().create()

# Set up Juman tokenizer
juman = Juman()

# Set up FastText model
model = fasttext.load_model('cc.ja.300.bin')

def preprocess(text):
    # Tokenize the text using MeCab
    tokens = mecab.parse(text).split()
    
    # Tokenize the text using SudachiPy
    sudachi_tokens = sudachi.tokenize(text)
    sudachi_tokens = [morph.surface() for morph in sudachi_tokens]
    
    # Tokenize the text using Juman
    juman_tokens = juman.analysis(text)
    juman_tokens = [mrph.midasi for mrph in juman_tokens.mrph_list()]
    
    # Combine the tokens from all tokenizers
    tokens += sudachi_tokens
    tokens += juman_tokens
    tokens = list(set(tokens))  # Remove duplicates
    
    # Remove stop words and punctuation
    stopwords = ['は', 'を', 'に', 'が', 'の', 'て', 'と', 'た', 'し', 'する', 'れる', 'さ', 'など']
    tokens = [token for token in tokens if token not in stopwords and not re.match(r'[。、・！？（）「」『』【】〈〉“”’‘｜\n]+', token)]

    # Insert spaces between Kanji and Hiragana/Katakana
    text = re.sub(r'(?<=[一-龯])(?=[ぁ-んァ-ン])', ' ', text)
    
    # Normalize the text
    text = sudachi.normalize(text)
    
    # Encode the words using FastText embedding
    vectors = [model.get_word_vector(word) for word in tokens]
    
    return vectors


# Look into this:
https://huggingface.co/sonoisa/t5-base-japanese

Moana is attmepting to use the same model too.. See what we can both come up with

# Trying out this model ... lots of big imports to install taking a long time...

https://tsmatz.wordpress.com/2022/11/25/huggingface-japanese-summarization/


https://github.com/huggingface/transformers/blob/main/src/transformers/models/t5/modeling_t5.py

https://github.com/huggingface/transformers/issues/4092

In [ ]:
import tensorflow
from transformers import AutoTokenizer, AutoModel

# tokenize text into ids
tokenizer = AutoTokenizer.from_pretrained("sonoisa/t5-base-japanese")

model = AutoModel.from_pretrained("sonoisa/t5-base-japanese")

In [ ]:
from transformers import pipeline

# sequence to id
seq2seq = pipeline("summarization", model="tsmatz/mt5-summarize-jp")
sample_text = "サッカーのワールドカップカタール大会、世界ランキング24位でグループEに属する日本は、23日の1次リーグ初戦において、世界11位で過去4回の優勝を誇るドイツと対戦しました。試合は前半、ドイツの一方的なペースではじまりましたが、後半、日本の森保監督は攻撃的な選手を積極的に動員して流れを変えました。結局、日本は前半に1点を奪われましたが、途中出場の堂安律選手と浅野拓磨選手が後半にゴールを決め、2対1で逆転勝ちしました。ゲームの流れをつかんだ森保采配が功を奏しました。"
result = seq2seq(sample_text)
print(result)

In [ ]:
# loading model and data collator
import torch
from transformers import AutoConfig, TFAutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# see https://huggingface.co/docs/transformers/main_classes/configuration
mt5_config = AutoConfig.from_pretrained(
  "google/mt5-small",
  max_length=128,
  length_penalty=0.6,
  no_repeat_ngram_size=2,
  num_beams=15,
)
model = (TFAutoModelForSeq2SeqLM
         .from_pretrained("google/mt5-small", config=mt5_config))
         # .to(device))

In [ ]:
from transformers import AutoTokenizer
t5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

In [ ]:
def tokenize_sample_data(data):
  # Max token size is 14536 and 215 for inputs and labels, respectively.
  # Here I restrict these token size.
  input_feature = t5_tokenizer(data["text"], truncation=True, max_length=1024)
  label = t5_tokenizer(data["summary"], truncation=True, max_length=128)
  return {
    "input_ids": input_feature["input_ids"],
    "attention_mask": input_feature["attention_mask"],
    "labels": label["input_ids"],
  }

tokenized_df = df.map(
  tokenize_sample_data,
  remove_columns=["id", "url", "title", "summary", "text"],
  batched=True,
  batch_size=128)

tokenized_df

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
  t5_tokenizer,
  model=model,
  return_tensors="pt")

In [ ]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
  encoded_arg = t5_tokenizer(arg)
  return t5_tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, t5_tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  # (Note : Please change this code, when you perform on other languages except for Japanese)
  text_preds = [(p if p.endswith(("!", "！", "?", "？", "。")) else p + "。") for p in text_preds]
  text_labels = [(l if l.endswith(("!", "！", "?", "？", "。")) else l + "。") for l in text_labels]
  sent_tokenizer_jp = RegexpTokenizer(u'[^!！?？。]*[!！?？。]')
  text_preds = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(p))) for p in text_preds]
  text_labels = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(l))) for l in text_labels]
  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

In [ ]:
from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
  tokenized_df["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=5)
for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      num_beams=15,
      num_return_sequences=1,
      no_repeat_ngram_size=1,
      remove_invalid_values=True,
      max_length=128,
    )
  labels = batch["labels"]
  break

metrics_func([preds, labels])

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
  output_dir = "mt5-summarize-ja",
  log_level = "error",
  num_train_epochs = 10,
  learning_rate = 5e-4,
  lr_scheduler_type = "linear",
  warmup_steps = 90,
  optim = "adafactor",
  weight_decay = 0.01,
  per_device_train_batch_size = 2,
  per_device_eval_batch_size = 1,
  gradient_accumulation_steps = 16,
  evaluation_strategy = "steps",
  eval_steps = 100,
  predict_with_generate=True,
  generation_max_length = 128,
  save_steps = 500,
  logging_steps = 10,
  push_to_hub = False
)

In [ ]:
# trainer class that fine tunes the model
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
  model = model,
  args = training_args,
  data_collator = data_collator,
  compute_metrics = metrics_func,
  train_dataset = tokenized_ds["train"],
  eval_dataset = tokenized_ds["validation"].select(range(20)),
  tokenizer = t5_tokenizer,
)

trainer.train()